In [1]:
import sys
import os
from scipy import linalg
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
sys.path.append("..")
# Enable interactive plotting in Jupyter Notebook

from utils.camera_utils import *

In [2]:
run_name = "RUN3"

spiegel_calib = "spiegel.mp4"
front_calib = "front.mp4"
stereo_calib = "stereo.mp4"

In [3]:
calibration_dir = f"../videos/{run_name}/calibration"
detection_dir = f"../videos/{run_name}/detection"
detection_files = os.listdir(detection_dir)
sC = stereoCamera(name=run_name, camera_size={0: (480, 240), 1: (480, 240)})
vL = videoLoader()


# Setting the Camera Positions

In [ ]:
vL.load_video(os.path.join(detection_dir, detection_files[-1]))
frame = vL[10]
# 0 = mirror-cam 1 = frontal-cam
sC.set_anchor_point(frame, 0)
sC.set_anchor_point(frame, 1)
for frame in vL[:100]:
    frame = sC.draw_camera_region(frame)
    cv2.imshow("Frame", frame)
    frame0, frame1 = sC(frame)
    cv2.waitKey(10)
cv2.destroyAllWindows()


# Mirror Calibration

In [ ]:
vL.load_video(os.path.join(calibration_dir, spiegel_calib))
frames = vL[100:110]
sC.calibrate(frames,0, rows=8, columns=10)

# Frontal Calibration

In [ ]:
vL.load_video(os.path.join(calibration_dir, front_calib))
frames = vL[100:110]
sC.calibrate(frames,1, rows=7, columns=9)

# Stereo Calibration

In [4]:
vL.load_video(os.path.join(calibration_dir, stereo_calib))
frames = vL[100::20]

In [5]:
frames = [frames[-1]]

for i in range(2, 8, 2):
    print(os.path.join(calibration_dir, f"stereo{i}.mp4"))
    vL.load_video(os.path.join(calibration_dir, f"stereo{i}.mp4"))
    frames.append(vL[100])


../videos/RUN3/calibration/stereo2.mp4
../videos/RUN3/calibration/stereo4.mp4
../videos/RUN3/calibration/stereo6.mp4


In [ ]:
frames

In [7]:

sC.stereo_calibrate(frames, rows=8, columns=10, manual=True)

(720, 1280, 3) Debug, imgshape
Current line: [756, 288]
Current line: [756, 288, 730, 528]
Current line: [784, 528]
Current line: [784, 528, 808, 291]
Current line: [859, 290]
Current line: [859, 290, 840, 528]
Current line: [893, 529]
Current line: [893, 529, 910, 290]
Current line: [961, 288]
Current line: [961, 288, 949, 525]
Current line: [1005, 528]
Current line: [1005, 528, 1013, 291]
Current line: [1063, 291]
Current line: [1063, 291, 1059, 529]
Current line: [1113, 523]
Current line: [1113, 523, 1114, 290]
Current line: [1166, 289]
Current line: [1166, 289, 1169, 525]
Current line: [1216, 496]
Current line: [1216, 496, 688, 496]
Current line: [699, 465]
Current line: [699, 465, 1213, 466]
Current line: [1195, 435]
Current line: [1195, 435, 703, 432]
Current line: [705, 404]
Current line: [705, 404, 1202, 404]
Current line: [1204, 375]
Current line: [1204, 375, 711, 375]
Current line: [720, 343]
Current line: [720, 343, 1197, 344]
Current line: [1199, 316]
Current line: [1199, 3

# Save Calibration to YAML

In [ ]:
sC.save_to_yaml(f"{run_name}.yaml")

In [8]:
sC.save_to_yaml(f"{run_name}_exp.yaml")

In [6]:
sC.load_from_yaml(f"{run_name}.yaml")

In [ ]:
sC.conf["anchor_point"], sC.name